In [1]:
import tensorflow as tf
import numpy as np
import tfgraphviz as tfg
from tensorboard import notebook
from keras.models import Sequential 
from keras.layers import Activation, Dense 
tf.compat.v1.keras.layers.Dense

C:\Users\mferrero\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\mferrero\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\mferrero\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\mferrero\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWa

tensorflow.python.keras.layers.core.Dense

In [2]:
# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)
    
def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch

In [3]:
reset_graph()

n_inputs = 28*28 # MNIST
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

In [4]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)
X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]

In [5]:
reset_graph()

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")

with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, name="hidden1",
                              activation=tf.nn.relu)
    hidden2 = tf.layers.dense(hidden1, n_hidden2, name="hidden2",
                              activation=tf.nn.relu)
    logits = tf.layers.dense(hidden2, n_outputs, name="outputs")
    y_proba = tf.nn.softmax(logits)

with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")    

learning_rate = 0.01

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)
    
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [6]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [7]:
n_epochs = 50
batch_size = 50

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_test = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        print(epoch, "Train accuracy:", acc_train, "Test accuracy:", acc_test)
    
    save_path = saver.save(sess, "./my_model_finalv1.ckpt")

0 Train accuracy: 0.9 Test accuracy: 0.9028
1 Train accuracy: 0.92 Test accuracy: 0.9252
2 Train accuracy: 0.94 Test accuracy: 0.9372
3 Train accuracy: 0.9 Test accuracy: 0.942
4 Train accuracy: 0.94 Test accuracy: 0.9472
5 Train accuracy: 0.94 Test accuracy: 0.9512
6 Train accuracy: 1.0 Test accuracy: 0.9548
7 Train accuracy: 0.94 Test accuracy: 0.9612
8 Train accuracy: 0.96 Test accuracy: 0.962
9 Train accuracy: 0.94 Test accuracy: 0.965
10 Train accuracy: 0.92 Test accuracy: 0.9654
11 Train accuracy: 0.98 Test accuracy: 0.9672
12 Train accuracy: 0.98 Test accuracy: 0.9682
13 Train accuracy: 0.98 Test accuracy: 0.9704
14 Train accuracy: 1.0 Test accuracy: 0.9696
15 Train accuracy: 0.94 Test accuracy: 0.9716
16 Train accuracy: 0.98 Test accuracy: 0.9726
17 Train accuracy: 1.0 Test accuracy: 0.9726
18 Train accuracy: 0.98 Test accuracy: 0.9748
19 Train accuracy: 0.98 Test accuracy: 0.9756
20 Train accuracy: 1.0 Test accuracy: 0.975
21 Train accuracy: 1.0 Test accuracy: 0.9732
22 Train 

In [8]:
with tf.Session() as sess:
    saver.restore(sess, "./my_model_finalv1.ckpt") # or better, use save_path
    X_new_scaled = X_test[:20]
    Z = logits.eval(feed_dict={X: X_new_scaled})
    y_pred = np.argmax(Z, axis=1)

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from ./my_model_finalv1.ckpt


In [9]:
print("Predicted classes:", y_pred)
print("Actual classes:   ", y_test[:20])

Predicted classes: [7 2 1 0 4 1 4 9 5 9 0 6 9 0 1 5 9 7 3 4]
Actual classes:    [7 2 1 0 4 1 4 9 5 9 0 6 9 0 1 5 9 7 3 4]


In [10]:
g = tfg.board(tf.get_default_graph())
g.view()

'G.gv.pdf'

In [11]:
# Load the TensorBoard notebook extension
%load_ext tensorboard
#%reload_ext tensorboard

In [14]:
#%tensorboard --logdir ./
%tensorboard --logdir tf_logs
#!kill 14672

Reusing TensorBoard on port 6006 (pid 3080), started 5:55:57 ago. (Use '!kill 3080' to kill it.)

## Resposta:
Analisando o exemplo MLPWithTensorFlowLowLevelAPI.ipynb utilizando objetos da função neuron_layer() e utilizando objetos da classe dense() e não foi observada nenhuma diferença significativa de performance.